## Examples of usage

In [1]:
import numpy as np
import random
import pandas as pd
import meliora.meliora.val_tests as vt

In [2]:
# Import data
test_data_main = pd.read_csv('pd_test_data_main.csv')
test_data_add = pd.read_csv('pd_test_data_period_2.csv')

In [3]:
# Binomial test
vt.binomial_test(test_data_main, "ratings", "default_flag", "predicted_pd")

,Rating class,Predicted PD,Total count,Defaults,Actual Default Rate,p_value,Reject H0
0,A,0.103429,406,55,0.135468,0.023892,True
1,B,0.099524,480,41,0.085417,0.867441,False
2,C,0.095719,114,10,0.087719,0.660553,False


In [4]:
# Jeffrey's test
vt.jeffreys_test(test_data_main, 'ratings', 'default_flag', 'predicted_pd', 0.05)

,Rating class,Predicted PD,Total count,Defaults,Actual Default Rate,p_value,Reject H0
0,A,0.103429,406,55,0.135468,0.019959,True
1,B,0.099524,480,41,0.085417,0.849552,False
2,C,0.095719,114,10,0.087719,0.598649,False


In [5]:
# Brier score
vt.brier_score(test_data_main, "ratings", "default_flag", "predicted_pd")

0.0012895084997917349

In [6]:
# Hosmer-Lemeshow test
vt.hosmer_test(test_data_main, "ratings", "default_flag", "predicted_pd")

[0.1302500370047408, False]

In [7]:
# Spiegelhalter test
vt.spiegelhalter_test(test_data_main, "ratings", "default_flag", "predicted_pd")

(-0.6637590511485174, False)

In [10]:
# Spiegelhalter test
vt.accuracy_ratio(test_data_main, "predicted_pd", "default_flag")

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [9]:
test_data_main['default_flag'].unique()

array([0, 1], dtype=int64)

In [11]:
test_data_main['predicted_pd'].unique()

array([1.37214683e-01, 1.30364572e-01, 1.88543538e-01, 1.78812469e-01,
       1.35670066e-01, 1.40673410e-01, 9.74792567e-02, 1.62066885e-01,
       8.96951214e-02, 1.34310225e-01, 3.80126817e-02, 4.43156418e-02,
       1.10617980e-01, 1.87843940e-02, 0.00000000e+00, 2.96504058e-02,
       8.81754748e-02, 9.52385102e-02, 4.24603973e-02, 1.46009440e-01,
       1.80177676e-01, 1.53239152e-01, 1.11715906e-01, 1.17288447e-01,
       4.91559033e-02, 1.02120366e-01, 1.37118629e-01, 4.08739453e-02,
       1.38932566e-01, 2.03073581e-01, 1.59676960e-01, 2.41076218e-01,
       9.51282663e-02, 5.61903312e-02, 7.67962983e-02, 1.23062124e-01,
       6.72544150e-02, 3.63846915e-03, 2.20205337e-02, 7.21733288e-04,
       9.26798498e-02, 1.09096239e-01, 1.10352572e-01, 1.10684050e-01,
       7.14891314e-02, 1.11044751e-01, 2.85040886e-02, 1.37796053e-01,
       8.72940603e-02, 7.14489343e-02, 1.69167872e-01, 1.38241648e-01,
       9.48597582e-02, 1.09232645e-02, 1.53667744e-01, 1.12246004e-01,
      

In [ ]:
# Transform input data into the required format
df = test_data_main.groupby('ratings').agg({'predicted_pd': "mean", 'default_flag': ["count", "sum", "mean"]})
df.columns = ["PD", "N", "D", "Default Rate"]


In [ ]:
realised_values = df["Default Rate"]
predicted_values = df["PD"]

In [ ]:
from scipy.stats import norm

# Calculate mean squared error
errors = realised_values - predicted_values
mse = (errors**2).sum() / len(errors)

# # Calculate null expectation and variance of MSE
expectations = sum(predicted_values * (1 - predicted_values)) / len(realised_values)
variances = sum(predicted_values * (1 - 2 * predicted_values)**2 * (1 - predicted_values)) / len(realised_values)**2

# Calculate standardized statistic
z_score = (mse - expectations) / np.sqrt(variances)  # todo: check formula

# Calculate standardized MSE as test statistic, then its p-value
outcome = z_score > norm.ppf(1 - 0.05/2)

In [ ]:
errors

In [ ]:
mse

In [ ]:
norm.ppf(1 - 0.05/2)

In [ ]:
test_data_main.head()

In [ ]:
# hosmer_lemeshow = function(y, prob, g, stat_type){
#   mtx = cbind(y, y_not = 1- y, prob, prob_not = 1-prob)
#   mtx = as.data.frame(mtx)
#   mtx = mtx[order(mtx$prob),]
#   n <- length(prob)/g
#   nr <- nrow(mtx)
  
#   ## C statistics, same number of instances in each bin
#   if (stat_type == 'C'){
#     split_mtx = split(mtx, rep(1:ceiling(nr/n), each=n, length.out=nr))
#   }else{ ### H statistics, equal intervals
#     split_mtx = split(mtx, cut(mtx$prob, seq(0,1,1/g), include.lowest=TRUE))
#     split_mtx = split_mtx[sapply(split_mtx, nrow)>0]
#     ###
#   }
  
#   H_stat = 0
#   for (i in 1:length(split_mtx)){
#     obs = sum(split_mtx[[i]]$y == 1)
#     exp = sum(split_mtx[[i]]$prob)
#     obs_not = sum(split_mtx[[i]]$y == 0)
#     exp_not = sum(split_mtx[[i]]$prob_not)
    
#     if (exp == 0 || exp_not == 0){
#       next
#     }
    
#     bin_sum = ((obs - exp)**2)/exp + ((obs_not - exp_not)**2)/exp_not

#     H_stat = H_stat + bin_sum
#   }
#   PVAL = 1 - pchisq(H_stat, g - 2)
  
#   cat('PVALUE', PVAL, '\n')
#   cat('stat', H_stat, '\n')
#   return(PVAL)
# }

In [ ]:
# Spiegelhalter_z = function(y, prob){
#   alpha = 0.05
#   z_score = sum((y-prob)*(1-2*prob))/sqrt(sum(((1-2*prob)^2)*prob*(1-prob)))
#   print(z_score)
#   if (abs(z_score) > qnorm(1-alpha/2)){
#     print('reject null. NOT calibrated')
#   } else{
#     print('fail to reject. calibrated')
#   }
#   cat('z score: ', z_score, '\n')
#   cat('p value: ', 1-pnorm(abs(z_score)), '\n')
#   return(z_score)
# }

In [ ]:
# https://documentation.sas.com/doc/en/pgmsascdc/v_011/statug/statug_code_logiex19.htm

In [ ]:
# ##Entropy
# def entropy(Y):
#     """
#     Also known as Shanon Entropy
#     Reference: https://en.wikipedia.org/wiki/Entropy_(information_theory)
#     """
#     unique, count = np.unique(Y, return_counts=True, axis=0)
#     prob = count/len(Y)
#     en = np.sum((-1)*prob*np.log2(prob))
#     return en


# #Joint Entropy
# def jEntropy(Y,X):
#     """
#     H(Y;X)
#     Reference: https://en.wikipedia.org/wiki/Joint_entropy
#     """
#     YX = np.c_[Y,X]
#     return entropy(YX)

# #Conditional Entropy
# def cEntropy(Y, X):
#     """
#     conditional entropy = Joint Entropy - Entropy of X
#     H(Y|X) = H(Y;X) - H(X)
#     Reference: https://en.wikipedia.org/wiki/Conditional_entropy
#     """
#     return jEntropy(Y, X) - entropy(X)


# #Information Gain
# def gain(Y, X):
#     """
#     Information Gain, I(Y;X) = H(Y) - H(Y|X)
#     Reference: https://en.wikipedia.org/wiki/Information_gain_in_decision_trees#Formal_definition
#     """
#     return entropy(Y) - cEntropy(Y,X)

In [ ]:
# https://machinelearningmastery.com/divergence-between-probability-distributions/

In [ ]:
# https://medium.com/@monadsblog/the-kullback-leibler-divergence-5071c707a4a6